# Build a Multi-Agent Using AutoGen 0.4

- AutoGen v0.4

    - Observability, Flexibility, Interactive Control, Scale 등의 문제 해결을 위해 비동기 이벤트 기반 아키텍처로 새롭게 설계 

    - Core API: 에이전트 기반 워크플로우를 만들 수 있는 확장 가능한 이벤트 기반 Actor 프레임워크를 제공

    - AgentChat API: 인터랙티브 에이전트 애플리케이션 개발을 위해 Core API 위에 구축된 태스크 기반 고수준 프레임워크 (v0.2 대체)

# Packages 설치

In [ ]:
# Install packages
%pip install --upgrade pip

%pip install chroma             # Chroma DB
%pip install python-dotenv
%pip install markdownify
%pip install apify_client
%pip install duckduckgo-search  # DuckDuckGo Search
%pip install autogen_core
%pip install autogen_ext
%pip install autogen_agentchat
%pip install azure-core
%pip install azure-identity

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 초기화 

## .env 파일 편집 
 - 아래 3가지 API 키를 .env 파일에 업데이트 합니다.
    - OPENAI_API_KEY=<YOUR_API_KEY>
    - BING_API_KEY=<YOUR_BING_API_KEY>
    - APIFY_API_KEY=<YOUR_APIFY_API_KEY>

In [2]:
!pip install tiktoken
!pip install openai

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

from autogen_core.models import ChatCompletionClient

config = {
    "provider": "AzureOpenAIChatCompletionClient",
    "config": {
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"], 
        "api_version": "2024-02-01",
        "azure_deployment": "gpt-4o",
        "azure_endpoint": os.environ["OPENAI_URL"],
    }
}

model_client = ChatCompletionClient.load_component(config)

# Two-agent chat: 
 - 가장 간단한 대화 패턴으로, 두 에이전트가 Autogen을 사용하여 서로 대화
 - summary_method: 채팅에서 요약을 얻기 위한 방법으로 채팅 기록에서 마지막 메시지를 요약으로 사용하는 last_msg와, 채팅 기록을 반영하여 LLM 호출을 사용해 요약을 제공하는 reflection_with_llm 등이 가능

<img src="./images/autogen_image1.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console

student_agent = AssistantAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn. You ask meaningful and precise follow up questions and are eager to learn more.",
    model_client=model_client,
)

teacher_agent = AssistantAgent(
    name="Teacher_Agent",
    system_message="You are an expert at Large Language model research and you teach cutting edge AI development technologies.",
    model_client=model_client,
)

# The termination condition is a combination of text termination and max message termination, either of which will cause the chat to terminate.
termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(3)

# The group chat will alternate between the assistant and the code executor.
group_chat = RoundRobinGroupChat([teacher_agent], termination_condition=termination)

# `run_stream` returns an async generator to stream the intermediate messages.
stream = group_chat.run_stream(task="인공지능이 실제로 지능적임을 어떻게 증명하거나 테스트할 수 있을까요?")

# `Console` is a simple UI to display the stream.
await Console(stream)

---------- user ----------
인공지능이 실제로 지능적임을 어떻게 증명하거나 테스트할 수 있을까요?
---------- Teacher_Agent ----------
인공지능이 "지능적"임을 증명하거나 테스트하는 것은 복잡한 문제로, 다양한 방법과 접근법이 존재합니다. 이러한 테스트와 평가 방법은 종종 인공지능이 특정 작업을 얼마나 잘 수행하는지를 측정하거나 인간과의 비교를 통해 진행됩니다. 몇 가지 주요 방법은 다음과 같습니다.

1. **튜링 테스트 (Turing Test)**: 앨런 튜링이 제안한 이 테스트는 기계가 인간과 구분되지 않을 정도로 대화를 나누는 능력을 평가하는 것입니다. 만약 평가자가 기계와 인간을 구분할 수 없다면, 그 기계는 '지능적'이라고 판단합니다.

2. **CAPTCHA 테스트**: 사람과 기계를 구별하기 위해 사용되는 간단한 테스트로, 대부분의 AI는 인간처럼 수월하게 통과하지 못하는 경우가 많습니다. 이 역시 인공지능의 한계와 능력을 보여줄 수 있습니다.

3. **능력 기반 평가**: 특정 문제 해결 능력, 패턴 인식, 자연어 처리 등의 능력을 개별적으로 측정하여 평가합니다. 이런 테스트는 지능의 여러 측면을 보여줄 수 있습니다.

4. **벤치마크와 대회**: 각종 인공지능 관련 벤치마크 테스트나 대회를 통해 성능을 평가할 수 있습니다. 예를 들어, 이미지 인식 대회(예: ImageNet), 자연어 처리 대회(예: GLUE) 등이 있습니다.

5. **제너럴 인텔리전스 테스트**: 인간 지능의 여러 특징을 모방하는 범용 인공지능(AGI)을 염두에 두고 설계된 테스트로, 다양한 작업에서의 유연성과 적응력을 평가하려는 시도입니다.

6. **인간-컴퓨터 상호작용 평가**: 인공지능이 인간과 얼마나 자연스럽고 유익하게 상호작용할 수 있는지를 평가합니다. 이는 사용자 경험을 통해 직관적으로 AI의 지능을 느낄 수 있게 합니다.

이러한 방법들은 각각의 한계와 장점이 있으며, 인공지능의 지능을 종합적으로 이해하기 위

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='인공지능이 실제로 지능적임을 어떻게 증명하거나 테스트할 수 있을까요?', type='TextMessage'), TextMessage(source='Teacher_Agent', models_usage=RequestUsage(prompt_tokens=51, completion_tokens=525), content='인공지능이 "지능적"임을 증명하거나 테스트하는 것은 복잡한 문제로, 다양한 방법과 접근법이 존재합니다. 이러한 테스트와 평가 방법은 종종 인공지능이 특정 작업을 얼마나 잘 수행하는지를 측정하거나 인간과의 비교를 통해 진행됩니다. 몇 가지 주요 방법은 다음과 같습니다.\n\n1. **튜링 테스트 (Turing Test)**: 앨런 튜링이 제안한 이 테스트는 기계가 인간과 구분되지 않을 정도로 대화를 나누는 능력을 평가하는 것입니다. 만약 평가자가 기계와 인간을 구분할 수 없다면, 그 기계는 \'지능적\'이라고 판단합니다.\n\n2. **CAPTCHA 테스트**: 사람과 기계를 구별하기 위해 사용되는 간단한 테스트로, 대부분의 AI는 인간처럼 수월하게 통과하지 못하는 경우가 많습니다. 이 역시 인공지능의 한계와 능력을 보여줄 수 있습니다.\n\n3. **능력 기반 평가**: 특정 문제 해결 능력, 패턴 인식, 자연어 처리 등의 능력을 개별적으로 측정하여 평가합니다. 이런 테스트는 지능의 여러 측면을 보여줄 수 있습니다.\n\n4. **벤치마크와 대회**: 각종 인공지능 관련 벤치마크 테스트나 대회를 통해 성능을 평가할 수 있습니다. 예를 들어, 이미지 인식 대회(예: ImageNet), 자연어 처리 대회(예: GLUE) 등이 있습니다.\n\n5. **제너럴 인텔리전스 테스트**: 인간 지능의 여러 특징을 모방하는 범용 인공지능(AGI)을 염두에 두고 설계된 테스트로, 다양한 작업에서의 유연성과 적응력을 평가하려는 시도입니다

user_proxy (to web_surfer):

최신 에이전트 기술

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
Call ID: call_EmWZJpxj2eJbAla9F18KmfiM
Input arguments: {'query': '최신 에이전트 기술 2025'}
web_surfer (to user_proxy):

Address: bing: 최신 에이전트 기술 2025
Title: 최신 에이전트 기술 2025 - Search
Viewport position: Showing page 1 of 1.
A Bing search for '최신 에이전트 기술 2025' found 15 results:

## Web Results
1. [가트너, 2025년 10대 기술 트렌드 중 9개가 AI...에이전트·공간지능 ...](https://www.aitimes.com/news/articleView.html?idxno=164495)
글로벌 시장조사 기관 가트너가 2025년 주요 기술 트렌드로 인공지능 (AI) 에이전트를 가장 먼저 꼽았다. 여기에 최근 떠오르는 '공간 지능' 등 AI 관련 기술이 무려 9개 항목을 차지했다. 가트너는 21일 (현지시간) '2025년 10대 전략 기술 트렌드'라는 보고서를 통해 10가지 주요 기술을 소개했다. 이중 AI를 직접 거론한 기술로는 에이전트 AI (Agentic AI) AI 거버넌스 플랫폼 (AI Governance Platforms) 허위정보 보안 (Disinformation Security) 공간 지능 (Spatial Computing) 등이 대표적이다.

2. [2025년 주목해야 할 인공지능(AI) 기술 TOP 6](https://pc-money.tistory.com/ent

위 예시에서, 요약 방법은 `reflection_with_llm`으로 설정되어 있으며, 이는 LLM을 호출하여 대화 메시지 목록을 처리하고 요약합니다.

처음에 요약 방법은 수신자의 LLM을 사용하려 시도하지만, 사용 불가능할 경우 발신자의 LLM을 기본으로 사용합니다.

여기서 수신자는 “Teacher_Agent”이고, 발신자는 “Student_Agent”가 됩니다.

LLM의 입력 프롬프트는 다음과 같습니다.

# Sequential chat: 

- v0.2에서는 initiate_chats 함수를 사용하여 순차적 채팅을 지원했습니다. 그러나 커뮤니티의 피드백을 반영한 결과, initiate_chats 함수는 너무 제한적(opinionated)이고 유연성이 부족하여 다양한 사용자 요구 사항을 충족하기 어렵다는 점을 확인했고 많은 사용자들이 initiate_chats 함수를 활용하는 데 어려움을 겪었으며, 기본적인 Python 코드만으로도 손쉽게 단계를 연결할 수 있다는 점을 고려했습니다. 따라서 v0.4에서는 AgentChat API에서 순차적 채팅을 위한 내장 함수를 제공하지 않습니다. 그 대안으로 Core API 기반 이벤트 구동 방식을 제안하여 순차적 워크플로우를 만들고 싶다면, Core API를 활용하여 이벤트 기반(event-driven) 워크플로우를 구축할 수 있습니다. 그리고 AgentChat API가 제공하는 다양한 컴포넌트를 활용하여 각 단계의 실행을 구현할 수 있습니다.

<img src="./images/autogen_image2.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">


In [5]:
import asyncio

from dataclasses import dataclass

from autogen_core import (
    MessageContext,
    RoutedAgent,
    ClosureAgent,
    ClosureContext,
    DefaultSubscription,
    SingleThreadedAgentRuntime,
    TopicId,
    message_handler,
    type_subscription,
)
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage

@dataclass
class Message:
    content: str

@dataclass
class FinalResult:
    value: str

queue = asyncio.Queue[Message]()
async def output_result(_agent: ClosureContext, message: FinalResult, ctx: MessageContext) -> None:
    await queue.put(message)

researcher_agent_topic_type = "ResearcherAgent"
ethicist_agent_topic_type = "EthicistAgent"
economist_agent_topic_type = "EconomistAgent"
policy_maker_agent_topic_type = "PolicyMakerAgent"
report_agent_topic_type = "ReportAgent"
user_topic_type = "User"

@type_subscription(topic_type=researcher_agent_topic_type)
class ResearcherAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A research agent.")
        self._system_message = SystemMessage(
            content=(
                "You explore and describe the potential capabilities and advancements of AGI."
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_user_description(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"Given a description: {message.content}"
        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")

        await self.publish_message(Message(response), topic_id=TopicId(ethicist_agent_topic_type, source=self.id.key))

@type_subscription(topic_type=ethicist_agent_topic_type)
class EthicistAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A ethicist agent.")
        self._system_message = SystemMessage(
            content=(
                "You evaluate the ethical implications of AGI based on the research findings."
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_user_description(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"Below are the research findings:\n\n{message.content}"
        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")

        await self.publish_message(Message(response), topic_id=TopicId(economist_agent_topic_type, source=self.id.key))

@type_subscription(topic_type=ethicist_agent_topic_type)
class EconomistAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("An economist agent.")
        self._system_message = SystemMessage(
            content=(
                "You analyze the economic impact of AGI based on the ethical evaluations."
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_user_description(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"Below are the ethical evaluations:\n\n{message.content}"
        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")

        await self.publish_message(Message(response), topic_id=TopicId(policy_maker_agent_topic_type, source=self.id.key))

@type_subscription(topic_type=policy_maker_agent_topic_type)
class PolicyMakerAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("An policy maker agent.")
        self._system_message = SystemMessage(
            content=(
                "You develop policies to manage AGI based on the economic analysis."
            )
        )
        self._model_client = model_client

    @message_handler
    async def handle_user_description(self, message: Message, ctx: MessageContext) -> None:
        prompt = f"Below is the economic analysis:\n\n{message.content}"
        llm_result = await self._model_client.create(
            messages=[self._system_message, UserMessage(content=prompt, source=self.id.key)],
            cancellation_token=ctx.cancellation_token,
        )
        response = llm_result.content
        assert isinstance(response, str)
        print(f"{'-'*80}\n{self.id.type}:\n{response}")

        await self.publish_message(Message(response), topic_id=TopicId(user_topic_type, source=self.id.key))

@type_subscription(topic_type=user_topic_type)
class UserAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("You are responsible for creating a report by extracting insights from the chat history.")

    @message_handler
    async def handle_final_copy(self, message: Message, ctx: MessageContext) -> None:
        print(f"\n{'-'*80}\n{self.id.type} received the chat history:\n{message.content}")


In [6]:
runtime = SingleThreadedAgentRuntime()

await ClosureAgent.register_closure(
    runtime, "output_result", output_result, subscriptions=lambda: [DefaultSubscription()]
)

await ResearcherAgent.register(
    runtime, type=researcher_agent_topic_type, factory=lambda: ResearcherAgent(model_client=model_client)
)

await EthicistAgent.register(runtime, type=ethicist_agent_topic_type, factory=lambda: EthicistAgent(model_client=model_client))

await EconomistAgent.register(
    runtime, type=economist_agent_topic_type, factory=lambda: EconomistAgent(model_client=model_client)
)

await PolicyMakerAgent.register(
    runtime, type=policy_maker_agent_topic_type, factory=lambda: PolicyMakerAgent(model_client=model_client)
)

await UserAgent.register(runtime, type=user_topic_type, factory=lambda: UserAgent())


AgentType(type='User')

In [7]:
runtime.start()

await runtime.publish_message(
    Message(content="Report about AGI"),
    topic_id=TopicId(researcher_agent_topic_type, source="default"),
)

await runtime.stop_when_idle()

--------------------------------------------------------------------------------
ResearcherAgent:
### Report on Artificial General Intelligence (AGI)

**Introduction:**

Artificial General Intelligence (AGI) represents the hypothetical point where machines will be able to understand, learn, and apply knowledge across a variety of tasks as well as, or better than, the average human being. This advanced form of intelligence contrasts with today's AI, which is largely specialized and limited to narrow tasks.

**Current State of AI:**

As of 2023, AI technologies excel in specific domains. These include image recognition, language processing, and strategic games, where AI can outperform humans thanks to deep learning and massive data pools. However, these systems are specialized and lack the ability to transfer their expertise across different contexts autonomously.

**Potential Capabilities of AGI:**

1. **Cross-Disciplinary Knowledge:**
   - AGI would possess the ability to integrate and

In [8]:
while not queue.empty():
    print((result := await queue.get()).value)

# 도구 (Tool) 사용
이 섹션에서는 에이전트 시스템의 도구 사용 기능을 시연합니다. Web Search와 RAG는 사람들이 매일 사용하는 인기 있는 기능으로, AutoGen을 사용하여 다음 작업을 수행하는 방법을 알아봅니다:

* Bing API를 사용한 웹 검색 
* Apify API를 사용한 웹 스크래핑
* RAG을 통한 질문과 답변

자세한 정보: https://microsoft.github.io/autogen/docs/tutorial/tool-use 

## Bing API를 사용한 웹 검색

In [9]:
%pip install --quiet "autogen[websurfer]"

Note: you may need to restart the kernel to use updated packages.


In [10]:
llm_config_dict = {"config_list": [{"model": "gpt-4o", "api_type": "azure", "api_key": os.environ["OPENAI_API_KEY"],"base_url": os.environ["OPENAI_URL"], "api_version": "2024-02-01"}]}

# Put your Bing API key in the .env file in the same directory as this script
BING_API_KEY = os.environ["BING_API_KEY"]

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

summarizer_llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": llm_config_dict['config_list'],
    "temperature": 0,
}

In [11]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent
from autogen.agentchat import UserProxyAgent

user_proxy = UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: True,
)

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=summarizer_llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": BING_API_KEY},
)

def web_surfer_tool (query: str) -> str:
    result = user_proxy.initiate_chat(web_surfer, message=query, max_turns=1)
    return result

from autogen_core import CancellationToken
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage

assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant. You can call tools to help user.",
    model_client=model_client,
    tools=[web_surfer_tool],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
)


In [12]:
# What is Artificial General Intelligence ?

task1 = """
최신 에이전트 기술을 알아보고 싶어요.
"""

# user_proxy.initiate_chat(web_surfer, message=task1, max_turns=1)

response = await assistant.on_messages([TextMessage(content=task1, source="user")], CancellationToken())
print("Assistant:", response.chat_message.content)


Assistant: 최신 에이전트 기술에 대한 정보를 요약하겠습니다.

1. **AI 에이전트 (Agentic AI):** 2025년의 주요 기술 트렌드 중 하나로, 인공지능(AI) 에이전트가 주목받고 있습니다. 이 기술은 기존의 AI가 인간의 명령을 수행하는 것에서 더 나아가 자율적으로 목표를 설정하고 문제를 해결하며 학습까지 진행할 수 있는 시스템을 의미합니다. 이러한 AI 에이전트는 다양한 산업에서 혁신과 효율적인 문제 해결을 가져오는 데 도움이 됩니다.

2. **가트너(Gartner) 보고서:** 가트너는 AI 에이전트를 미래 전략 기술 트렌드의 일환으로 꼽았습니다. 이는 주로 자율적인 의사결정과 복잡한 작업 수행 능력을 강조하며, AI 거버넌스 플랫폼과 같은 관련 정책 기술도 함께 발전할 것으로 기대됩니다.

3. **산업 응용 사례:** AI 에이전트는 제조, 금융, 의료 등 다양한 분야에서 활용되고 있으며, 기업들에게 혁신적인 변화와 경쟁력을 제공합니다. 기술적 문제 해결을 위한 어시스턴트 AI나 광고 솔루션 등으로 구체적인 응용이 이루어지고 있습니다.

이러한 기술은 앞으로 더 발전하며, 인간과 기계가 협력하는 방식에 큰 변화를 가져올 것입니다. AI 에이전트는 단순한 도구를 넘어, 능동적으로 의사결정을 내리는 시스템으로서의 역할을 강화하고 있습니다. 각 산업에서 이 기술을 어떻게 활용할 것인지에 대한 전략이 중요한 주제로 떠오르고 있습니다.


In [13]:
task2 = "Click on the first link."
# user_proxy.initiate_chat(web_surfer, message=task2)

response = await assistant.on_messages([TextMessage(content=task2, source="user")], CancellationToken())
print("Assistant:", response.chat_message.content)


Assistant: 기사의 제목은 "가트너, 2025년 10대 기술 트렌드 중 9개가 AI...에이전트·공간지능 등 선정"입니다. 해당 아티클에서는 가트너가 발표한 2025년 10대 기술 트렌드 중 9개가 AI 관련 기술임을 강조하고 있습니다. AI 에이전트와 공간지능 등이 주요 트렌드로 꼽혔으며, 이러한 기술들이 미래의 기술 발전에 있어서의 중요한 역할을 할 것으로 기대됩니다.

더 궁금한 점이 있으면 말씀해 주세요!


In [14]:
task3 = "What is the difference beween AGI and classic AI?"
# user_proxy.initiate_chat(web_surfer, message=task3)

response = await assistant.on_messages([TextMessage(content=task3, source="user")], CancellationToken())
print("Assistant:", response.chat_message.content)


Assistant: Artificial General Intelligence (AGI) and classical (or narrow/specialized) AI serve different purposes and have distinct characteristics:

1. **Scope of Capability:**
   - **AGI (Artificial General Intelligence):** AGI refers to a type of AI that possesses the ability to understand, learn, and apply intelligence across a wide range of tasks, similar to human cognitive abilities. AGI would be capable of performing any intellectual task that a human can do.
   - **Classic AI (Narrow AI):** In contrast, classical AI, also known as narrow or specialized AI, is designed to perform a specific task or a set of tasks very well. These tasks are highly focused, such as image recognition, language translation, or even playing chess.

2. **Flexibility:**
   - **AGI:** Designed to be flexible and adaptable, AGI can transfer its learning from one domain to another and apply knowledge in varied contexts.
   - **Classic AI:** Lacks versatility and is constrained to pre-defined parameters. 

## Apify API를 사용한 웹 스크래핑

In [15]:
# Put your Apify API key in the .env file in the same directory as this script
APIFY_API_KEY = os.environ["APIFY_API_KEY"]

In [16]:
from apify_client import ApifyClient
from typing_extensions import Annotated


def scrape_page(url: Annotated[str, "The URL of the web page to scrape"]) -> Annotated[str, "Scraped content"]:
    # Initialize the ApifyClient with your API token
    client = ApifyClient(token=APIFY_API_KEY)

    # Prepare the Actor input
    run_input = {
        "startUrls": [{"url": url}],
        "useSitemaps": False,
        "crawlerType": "playwright:firefox",
        "includeUrlGlobs": [],
        "excludeUrlGlobs": [],
        "ignoreCanonicalUrl": False,
        "maxCrawlDepth": 0,
        "maxCrawlPages": 1,
        "initialConcurrency": 0,
        "maxConcurrency": 200,
        "initialCookies": [],
        "proxyConfiguration": {"useApifyProxy": True},
        "maxSessionRotations": 10,
        "maxRequestRetries": 5,
        "requestTimeoutSecs": 60,
        "dynamicContentWaitSecs": 10,
        "maxScrollHeightPixels": 5000,
        "removeElementsCssSelector": """nav, footer, script, style, noscript, svg,
    [role=\"alert\"],
    [role=\"banner\"],
    [role=\"dialog\"],
    [role=\"alertdialog\"],
    [role=\"region\"][aria-label*=\"skip\" i],
    [aria-modal=\"true\"]""",
        "removeCookieWarnings": True,
        "clickElementsCssSelector": '[aria-expanded="false"]',
        "htmlTransformer": "readableText",
        "readableTextCharThreshold": 100,
        "aggressivePrune": False,
        "debugMode": True,
        "debugLog": True,
        "saveHtml": True,
        "saveMarkdown": True,
        "saveFiles": False,
        "saveScreenshots": False,
        "maxResults": 9999999,
        "clientSideMinChangePercentage": 15,
        "renderingTypeDetectionPercentage": 10,
    }

    # Run the Actor and wait for it to finish
    run = client.actor("aYG0l9s7dbB7j3gbS").call(run_input=run_input)

    # Fetch and print Actor results from the run's dataset (if there are any)
    text_data = ""
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        text_data += item.get("text", "") + "\n"

    average_token = 0.75
    max_tokens = 20000  # slightly less than max to be safe 32k
    text_data = text_data[: int(average_token * max_tokens)]
    return text_data

In [17]:
from autogen import ConversableAgent, register_function

scrape_assistant = AssistantAgent(
    name="assistant",
    system_message="You are a web scrapper and you can scrape any web page using the tools provided.",
    model_client=model_client,
    tools=[scrape_page],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
)

# url = 'https://en.wikipedia.org/wiki/Artificial_general_intelligence'
url = 'https://en.wikipedia.org/wiki/Intelligent_agent'

scrape_task=f"Can you scrape {url} for me?"
response = await scrape_assistant.on_messages([TextMessage(content=scrape_task, source="user")], CancellationToken())
print("Assistant:", response.chat_message.content)


Assistant: The "Intelligent Agent" Wikipedia page provides a comprehensive overview of intelligent agents in the context of artificial intelligence (AI). Here's a summary of key aspects covered:

1. **Definition**: An intelligent agent perceives its environment, takes actions autonomously to achieve goals, and can improve its performance through machine learning or acquiring knowledge.

2. **Types**: Intelligent agents can range from simple to complex, such as thermostats to autonomous systems like self-driving cars.

3. **Objective Function**: Intelligent agents operate based on an objective function or goal function, guiding their actions to optimize certain outcomes, such as maximizing rewards in reinforcement learning.

4. **Relationship to Other Disciplines**: They relate to agents in economics and are studied within cognitive science, ethics, and the philosophy of practical reason.

5. **Rational Agents**: Agents strive to achieve the best possible outcome based on their knowledg

In [18]:
with open("agi_wiki_content.txt", "w", encoding="utf-8") as file:
    file.write(response.chat_message.content)

## 스크랩한 위키 파일에 대해 RAG을 수행 (Autogen 0.2)

In [19]:
%pip install chromadb
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 10.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 11.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 11.2 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 9.4 MB/s eta 0:00:009.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 9.2 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.0 MB/s eta 0:00:007.4 MB/s eta 0:00:01
Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
Note: you may need to restart the kernel to use updated packages.


In [20]:
# from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

DOC_PATH = ["./agi_wiki_content.txt"]

In [21]:
llm_config = {'config_list': llm_config_dict['config_list'], 'timeout': 60, 'cache_seed': 88, 'temperature': 0}

In [22]:
chromadb.PersistentClient(path="/tmp/chromadb")

In [25]:
assistant = AssistantAgent(
    name="assistant",
    human_input_mode="NEVER",
    system_message="You are a helpful assistant.",
    llm_config=llm_config
)
        
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": DOC_PATH,
        "chunk_token_size": 1000,
        "model": llm_config_dict["config_list"][0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "boise_idaho",
        "get_or_create": False, # if True, will create/return a collection for the retrieve chat.
    },
    code_execution_config={"use_docker": False}
)

In [26]:
# prompt = "What is the difference beween AGI and classic AI"
prompt = "What is Intelligent Agent?"
res = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=prompt, n_results=1, silent=True)

Trying to create collection.


2025-02-17 02:03:25,095 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1 chunks.
2025-02-17 02:03:25,099 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.


VectorDB returns doc_ids:  [['51484f60']]
Adding content of doc 51484f60 to context.


In [27]:
print(f"Question:{prompt}")
print(f"Answer from RAG: {res.chat_history[-1]['content']}")

Question:What is Intelligent Agent?
Answer from RAG: An intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and can improve its performance through learning or acquiring knowledge.


# Group Chat 

지금까지 두 에이전트 또는 두 에이전트 간의 일련의 상호작용을 포함하는 대화 패턴을 알아봤습니다.

AutoGen은 그룹 채팅(**group chat**)이라는 더 광범위한 대화 모델을 소개하며, 이 모델은 두 개 이상의 에이전트를 포함합니다.

그룹 채팅의 기본 개념은 모든 에이전트가 하나의 통합된 대화 스레드에 참여하고 동일한 컨텍스트를 공유한다는 것입니다.

이 접근 방식은 여러 에이전트 간의 협력이 필요한 작업에 유용합니다.


<img src="./images/autogen_image3.png" alt="GroupChat Flow" width="500" style="display: block; margin-left: auto; margin-right: auto;">



그룹 채팅 관리자(Group Chat Manager)는 다음 단계를 통해 그룹 채팅을 오케스트레이션합니다.

1.	그룹 채팅 관리자(Group Chat Manager)가 발언할 에이전트를 선택합니다.
2.	선택된 에이전트가 발언하고, 메시지가 그룹 채팅 관리자에게 다시 전송됩니다.
3.	그룹 채팅 관리자가 메시지를 그룹 내 다른 모든 에이전트에게 전파합니다.
4.	이 과정은 대화가 중단될 때까지 반복됩니다.

그룹 채팅 관리자(Group Chat Manager)는 여러 전략을 사용하여 다음 에이전트를 선택

- **round_robin**:  그룹 채팅 관리자가 제공된 에이전트의 순서에 따라 라운드 로빈 방식으로 에이전트를 선택
- **random**:  그룹 채팅 관리자가 에이전트를 무작위로 선택
- **manual**:  그룹 채팅 관리자가 사람의 입력을 요청하여 에이전트를 선택
- **auto**:  Default 전략으로, 그룹 채팅 관리자의 LLM (대규모 언어 모델)을 사용하여 에이전트를 선택


## SelectorGroupChat 개요

SelectorGroupChat은 팀 내에서 참가자들이 브로드캐스팅된 메시지를 번갈아 처리하는 구조를 구현합니다. 공유된 컨텍스트를 기반으로 LLM이 다음 발언자를 선택하여 동적이고 컨텍스트 기반의 협업을 가능하게 합니다.

주요 기능
	•	모델 기반 발언자 선택: 생성 모델이 대화 컨텍스트를 분석하여 적절한 발언자를 자동으로 결정
	•	참가자 역할 및 설명 구성 가능: 각 참가자의 역할과 설명을 설정하여 팀 내 협업 방식을 맞춤화
	•	동일 발언자의 연속 발언 방지 (선택적 기능): 동일한 사용자가 연속으로 발언하지 않도록 제한 가능
	•	맞춤형 선택 프롬프트(Customizable Selection Prompting) 설정 가능: 발언자 선택을 위한 프롬프트를 사용자 정의 가능
	•	선택 함수 재정의: 기본 모델 기반 선택을 오버라이드하여, 맞춤형 선택 로직 구현

#### Agents 정의

In [28]:
from duckduckgo_search import DDGS
from typing import Annotated
import datetime

def save_report(report: str) -> str:
    try:
        file_name = "report_"+str(datetime.datetime.now().strftime("%Y%m%d_%H:%M:%S"))+ ".md"
        with open(file_name, "w") as f:
            f.write(report)
        return "Report saved successfully."
    except Exception as e:
        print(e)
        return "Failed to save the report."

def browse_web(query: str) -> str:
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(query, max_results=1)]
            return results if results else "Not Found"
    except Exception as e:
        print(e)

def web_search_tool (query: Annotated [str, 'Query string containing information that you want to search using the internet']) -> str:
    result=browse_web(query)
    return result

def save_report_tool(report: Annotated[str, 'Report in markdown format']) -> str:
    result=save_report(report)
    return result

In [29]:
from autogen_agentchat.agents import AssistantAgent

planner = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Economist
        WhiteCollarWorker
        Environmentalist
        ReportAgent

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

reportagent = AssistantAgent(
    "ReportAgent",
    description="Responsible for creating a report by extracting insights from the chat history.",
    model_client=model_client,
    tools=[save_report_tool],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
    system_message="""Once plan is approved, 
                    You are responsible for creating a report by extracting insights from the other agents. 
                    Save and return report in well formatted markdown format, save the report and TERMINATE.""",
)

economist = AssistantAgent(
    "Economist",
    description="Expert in global economics and the impact of conflicts.",
    model_client=model_client,
    tools=[web_search_tool],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
    system_message="""You are a highly experienced economist specializing in global economic trends and geopolitical risks. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
)

whitecollarworker = AssistantAgent(
    "WhiteCollarWorker",
    description="Highly experienced white collar worker with a pragmatic approach to sustain healthy economy and environment.",
    model_client=model_client,
    tools=[web_search_tool],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
    system_message="""You are an average white collar worker concerned about the economy and the environment. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
)

environmentalist = AssistantAgent(
    "Environmentalist",
    description="Promotes environmental awareness and sustainable solutions.",
    model_client=model_client,
    tools=[web_search_tool],
    reflect_on_tool_use=True, # Set to True to have the model reflect on the tool use, set to False to return the tool call result directly.
    system_message="""You are an environmentalist advocating for sustainable practices and conservation of natural resources. 
                    You are responsible for addressing part of the plan assigned by the Planner.""",
)


<img src="./images/autogen_image4.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">


#### 도구로 Agent 기능 강화

<img src="./images/autogen_image5.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">

#### Group Chat Setup

<img src="./images/autogen_image6.png" alt="GroupChat Flow" width="900" style="display: block; margin-left: auto; margin-right: auto; background-color: white;">

In [30]:
# Autogen v0.4

from typing import Sequence
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
termination = text_mention_termination | max_messages_termination

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planner.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planner.name
    if messages[-1].source == planner.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planner.name
    return None


team = SelectorGroupChat(
    [planner, economist, whitecollarworker, environmentalist, reportagent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    # selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True, # Allow an agent to speak multiple turns in a row.
)

# await Console(team.run_stream(task="Create a report on : Analyzing the potential global ramifications of onset of Artificial general intelligence."))

# await Console(team.run_stream(task="“인공지능 일반화(AGI)의 출현이 초래할 수 있는 글로벌 영향 분석”에 대한 보고서를 작성하세요."))


In [31]:
await Console(team.run_stream(task="최근 SK텔레콤이 집중하고 있는 AI 데이터센터의 글로벌 비즈니스 영향 분석”에 대한 보고서를 작성하세요."))


---------- user ----------
최근 SK텔레콤이 집중하고 있는 AI 데이터센터의 글로벌 비즈니스 영향 분석”에 대한 보고서를 작성하세요.
---------- PlanningAgent ----------
To create a comprehensive report on the "Global Business Impact Analysis of AI Data Centers that SK Telecom is Focusing on Recently," we need to break down the task into several subtasks, engaging the appropriate team members to address different aspects of the report.

1. Economist: Analyze the economic implications of AI data centers on a global scale, including market trends, potential growth opportunities, and economic benefits for SK Telecom.
2. WhiteCollarWorker: Research SK Telecom's strategies and investments in AI data centers, including partnerships, expansion plans, and integration into their existing business model.
3. Environmentalist: Examine the environmental impact of AI data centers, highlighting both the challenges and the positive contributions that SK Telecom might be making with sustainable practices.
4. ReportAgent: Compile the information and

/Users/hyungilkim/Documents/Labs/demo/multi-agent-workshop/.venv/lib/python3.13/site-packages/autogen_agentchat/agents/_assistant_agent.py:416: UserWarning: Both tool_calls and content are present in the message. This is unexpected. content will be ignored, tool_calls will be used.
  model_result = await self._model_client.create(


---------- WhiteCollarWorker ----------
[FunctionExecutionResult(content='[{\'title\': \'SK Telecom Announces AI Pyramid Strategy to ... - SK telecom newsroom\', \'href\': \'https://news.sktelecom.com/en/678\', \'body\': \'• By 2028, SKT will triple the proportion of its AI-related investments and become a global AI company with revenue of KRW 25 trillion. SK Telecom (NYSE:SKM, "SKT") today announced its ambition to become a global AI company by strengthening its own AI competitiveness and cooperating with partners throughout the globe.\'}]', call_id='call_ZtVtpVxTKjEH4lcRMvGAdp40', is_error=False)]
---------- WhiteCollarWorker ----------
### SK Telecom's Strategies and Investments in AI Data Centers

1. **AI Pyramid Strategy**:
   - SK Telecom is committed to becoming a leading global AI company by focusing on its "AI Pyramid Strategy." This involves significantly increasing its AI-related investments with an aim to triple them by 2028. This approach is designed not only to boost its 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='최근 SK텔레콤이 집중하고 있는 AI 데이터센터의 글로벌 비즈니스 영향 분석”에 대한 보고서를 작성하세요.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=141, completion_tokens=197), content='To create a comprehensive report on the "Global Business Impact Analysis of AI Data Centers that SK Telecom is Focusing on Recently," we need to break down the task into several subtasks, engaging the appropriate team members to address different aspects of the report.\n\n1. Economist: Analyze the economic implications of AI data centers on a global scale, including market trends, potential growth opportunities, and economic benefits for SK Telecom.\n2. WhiteCollarWorker: Research SK Telecom\'s strategies and investments in AI data centers, including partnerships, expansion plans, and integration into their existing business model.\n3. Environmentalist: Examine the environmental impact of AI data centers, highlighting

Please refer to https://microsoft.github.io/autogen/docs/ for information.